# Healthcare Services Location Optimization

## Introduction

In this notebook we aim to analyse a sample of the diagnosed diseases in Texas State and the respective hospitals that treat them. The user can specify various types of diagnosis in the notebook as well. Hospital and patient's data will be collected and an analysis of the cost of the hospital, its distance from the patient and patient demographics is carried out. <br>

In this notebook the user can download real, public patient, hospital and diagnosis data from various websites, transform it into a format for analysis and save the final datasets for visualisation using an Rshiny app. The accelerator already comes packaged with some synthetic data which was created to have similar fields to the publicly available real data. The user can use this data to quickly run through the notebook and visualisation before proceeding with the real data.

To use the real data, change the `data_source` variable to `real`. Use `synthetic` if using the synthetic data packaged with the project. 

**Sample Materials, provided under license. <br>
Licensed Materials - Property of IBM. <br>
© Copyright IBM Corp. 2019, 2020. All Rights Reserved. <br>
US Government Users Restricted Rights - Use, duplication or disclosure restricted by GSA ADP Schedule Contract with IBM Corp. <br>**

### Specify the data source

In [32]:
# specify if using data included in project (synthetic) or from the web (real)
# If 'real' is selected, the notebook downloads the data from the sources listed below

data_source = 'real' #synthetic/real

### Install the required libraries

In [33]:
!pip install fuzzywuzzy
!pip install geopy
!pip install python-Levenshtein 

In [34]:
import pandas as pd
import numpy as np
from fuzzywuzzy import process, fuzz
from geopy.distance import geodesic
import numpy as np
import os

# project-lib library contains functions that simplify fetching files associated with the project.
from project_lib import Project
project = Project()

## Data that can be downloaded


If the user selects to use the real data, it will be downloaded from the sources listed below and used for the analysis. The user may need to agree to some license terms listed later in this document in order to use this data.<br>

**1: Patients geographical information with latitude, longitude and zip codes   <br>**
Data Source: http://www2.census.gov/geo/docs/maps-data/data/gazetteer/2016_Gazetteer/2016_Gaz_zcta_national.zip  <br>
Column names:  <br>
PAT_ZIP, ALAND, AWATER,ALAND_SQMI, AWATER_SQMI, PAT_LAT,PAT_LONG <br>

**2: Hospital names with longitude and latitude  <br>**
Data Source: https://data.medicare.gov  <br>
Columns:  <br>
Long,	Lat, OBJECTID, NAME, STCTYFIPS, ELEV_METER  <br>


**3:  Diagnosis data in different quarters of the year.  <br>**
Data Source: https://www.dshs.texas.gov  <br>
License Agreement: https://www.dshs.texas.gov/THCIC/Hospitals/Download.shtm <br>
columns:  <br>
DISCHARGE_QTR, THCIC_ID, PROVIDER_NAME, ENCOUNTER_INDICATOR, SEX_CODE, TYPE_OF_ADMISSION, SOURCE_OF_ADMISSION, PAT_STATE, PAT_ZIP, PAT_COUNTRY*  <br>
*Column headers for the above datasets can be found at on https://www.dshs.state.tx.us/thcic/hospitals/UserManual_4q10.pdf 

### Lookups
Below lookup file is used to understand the diagnosis codes. <br>
**1: ICD-9 codes dataset  to match diagnosis  codes:** <br>
http://www.cms.gov/Medicare/Coding/ICD9ProviderDiagnosticCodes/Downloads/ICD-9-CM-v32-master-descriptions.zip

### Get Patients Geographical Information Data

If the user selects to pull the real data, the bash commands below download patient geographical information from the census website. Otherwise, we import a csv file with the synthetic patient geographical information.

In [35]:
if data_source == 'real':
    # Download the data from the  census website and extract
    !wget --quiet  --output-document 2016_Gaz_zcta_national.zip http://www2.census.gov/geo/docs/maps-data/data/gazetteer/2016_Gazetteer/2016_Gaz_zcta_national.zip
    !unzip -o 2016_Gaz_zcta_national.zip -d ZIP/

Archive:  2016_Gaz_zcta_national.zip
  inflating: ZIP/2016_Gaz_zcta_national.txt  


In [36]:
if data_source == 'real':
    df_patgeo = pd.read_csv("ZIP/2016_Gaz_zcta_national.txt", sep='\t')
    df_patgeo.columns = df_patgeo.columns.str.strip().str.upper().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
    df_patgeo= df_patgeo.rename(columns={"GEOID": 'PAT_ZIP', "INTPTLAT": 'PAT_LAT', "INTPTLONG": 'PAT_LONG'})

elif data_source == 'synthetic':
    

    zip_location_data_file = project.get_file("synthetic patients zip location data.csv")
    zip_location_data_file.seek(0)
    df_patgeo = pd.read_csv(zip_location_data_file)

df_patgeo = df_patgeo[['PAT_ZIP', 'PAT_LAT', 'PAT_LONG']]
df_patgeo.head()

,PAT_ZIP,PAT_LAT,PAT_LONG
0,601,18.180555,-66.749961
1,602,18.361945,-67.175597
2,603,18.455183,-67.119887
3,606,18.158345,-66.932911
4,610,18.295366,-67.125135


Patients geographical data contains following columns

**PAT_ZIP :** Patients zip code. <br>
**PAT_LAT :** Latitude corresponding to the zip code. <br>
**PAT_LONG :** Longitude corresponding to the zip code. <br>

### Get Hospital information data 

The data from the website https://data.medicare.gov/api/views/xubh-q36u/rows.csv?accessType=DOWNLOAD&sorting=true can be downloaded. If the synthetic data is being used instead, it is imported from a csv file using `project_lib` library. The file was created to have similar fields and content to the real data.<br> 
The dataset contains details on hospitals and their locations.

In [37]:
if data_source == 'real':
    # Below bash command will download the data and that will be stored in a dataframe dfHospital
    z=!wget -O hospital.csv https://data.medicare.gov/api/views/xubh-q36u/rows.csv?accessType=DOWNLOAD&sorting=true 
    dfHospital = pd.read_csv("hospital.csv", sep=',')
    # dfHospital dataset is cleaned to retrieve only texas state records
    dfHospital=dfHospital[dfHospital['State']=='TX']
elif data_source == 'synthetic':
    hospital_details_file = project.get_file("synthetic hospital details.csv")
    hospital_details_file.seek(0)
    dfHospital = pd.read_csv(hospital_details_file)

Extract longitude and latitude from the location.

Hospital dataset contains below columns <br>

**Hospital_Name:** Name of the hospital <br>
**longitude & latitude:** Coordinates of the hospital <br>

In [38]:
# seperate longitude and latitude columns and remove the additional characters
if data_source == 'real':
    dfHospital['longlat']=dfHospital['Location'].str.split("(").str[1].str.strip(')')
    dfHospital['longitude']=dfHospital['longlat'].str.split(' ').str[0]
    dfHospital['latitude']=dfHospital['longlat'].str.split(' ').str[1]
elif data_source == 'synthetic':
    dfHospital['longlat']=dfHospital['Location'].str.split("(").str[1].str.strip(')')
    dfHospital['longitude']=dfHospital['longlat'].str.split(',').str[1]
    dfHospital['latitude']=dfHospital['longlat'].str.split(',').str[0]
    
# Rename the columns 
dfHospital=dfHospital[['Facility ID','Facility Name','longitude','latitude']]
dfHospital.columns=['Provider_ID','Hospital_Name','longitude','latitude']
#Remove the columns with nulls
dfHospital=dfHospital[pd.notnull(dfHospital['longitude'])]
dfHospital.head()

,Provider_ID,Hospital_Name,longitude,latitude
15,670010,MAYHILL HOSPITAL,-97.088293,33.175842
34,450718,ROUND ROCK MEDICAL CENTER,-97.712361,30.509336
35,450193,CHI ST LUKE'S HEALTH BAYLOR COLLEGE OF MEDICIN...,-95.39918000000002,29.707116
62,450780,METHODIST HOSPITAL AMBULATORY SURGERY,-98.595995,29.527605000000005
65,450200,WADLEY REGIONAL MEDICAL CENTER,-94.047188,33.428482


### Get Patient Diagnosis data

The code below contains the link to an example of real world diagnosis data that can be used. The user should read and agree to the license terms at the following link before using the data. https://www.dshs.texas.gov/THCIC/Hospitals/Download.shtm. The user can then download the data themselves or uncomment and use the code in the cell below. The cells take a few minutes to run as the files downloaded are quite large.  

> The bash commands specified below are used to download the data. <br>
> Specify the quarter and year of the data that to be used for the analysis purpose. <br>
> Below set of code downloads the data for the years between 2008 and 2010. Once downloaded, these files will be extracted into ZIP folder. <br>
> Each PUDF_base*.txt file is processed and stored into a dataframe. Once the dataframe is loaded, all the files and zip folder will be deleted <br>

In [39]:
%%bash -s "$data_source"
if [ "$1" = "real" ]; then  # Download data from the above website, specify year and the diagnosis code

years=(1Q2008 2Q2008 3Q2008 4Q2008 1Q2009 2Q2009 3Q2009 4Q2009 1Q2010 2Q2010 3Q2010 4Q2010)
for year in "${years[@]}";
do
wget -O filename$year.zip https://www.dshs.texas.gov/thcic/hospitals/Data/PUDF-"$year"-tab-delimited/ --no-check-certificate --quiet
done
fi

In [40]:
if data_source == 'real':
    # Unzip the files and remove the unnecessary files from the disk, Once all the files are extracted remove the zip files too
    !unzip 'filename*.zip' -d ZIP/
    !rm -rf out*
    !rm -rf *.zip
    !rm -rf PUDF_charges*

Archive:  filename1Q2008.zip
  inflating: ZIP/PUDF_charges1q2008_tab.txt  
  inflating: ZIP/PUDF_base1q2008_tab.txt  

Archive:  filename2Q2008.zip
  inflating: ZIP/PUDF_charges2q2008_tab.txt  
  inflating: ZIP/PUDF_base2q2008_tab.txt  

Archive:  filename3Q2008.zip
  inflating: ZIP/PUDF_charges3q2008_tab.txt  
  inflating: ZIP/PUDF_base3q2008_tab.txt  

Archive:  filename4Q2008.zip
  inflating: ZIP/PUDF_charges4q2008_tab.txt  
  inflating: ZIP/PUDF_base4q2008_tab.txt  

Archive:  filename1Q2009.zip
  inflating: ZIP/PUDF_charges1q2009_tab.txt  
  inflating: ZIP/PUDF_base1q2009_tab.txt  

Archive:  filename2Q2009.zip
  inflating: ZIP/PUDF_charges2q2009_tab.txt  
  inflating: ZIP/PUDF_base2q2009_tab.txt  

Archive:  filename3Q2009.zip
   creating: ZIP/PUDF 3Q2009 Tab-delimited_2017/
  inflating: ZIP/PUDF 3Q2009 Tab-delimited_2017/PUDF_base3q2009_tab.txt  
  inflating: ZIP/PUDF 3Q2009 Tab-delimited_2017/PUDF_charges3q2009_tab.txt  

Archive:  filename4Q2009.zip
   creating: ZIP/PUDF 4Q200

After all of the files are downloaded we can process the PUDF_base*.txt files. <br>

Enter the diagnosis codes required for the user to analyse. Diagnosis codes can be found on the website **https://www.cms.gov/**
These diagnosis codes are in ICD-9 format.  International Classification of Diseases, Clinical Modification (ICD-9-CM) is an adaption created by the U.S. National Center for Health Statistics (NCHS) and used in assigning diagnostic and procedure codes associated with inpatient, outpatient, and physician office utilization in the United States.

We have used the diagnosis codes <b>[7453, 74510, 74569, 74511, 7450]</b> for this analysis. These diagnosis are cardiac related, which are usually found in children. (https://en.wikipedia.org/wiki/Bulbus_cordis)

In [41]:
if data_source == 'real':
    diagnosis_codes=["7453","74510","74569","74511","7450"]
    data=pd.DataFrame()

    columns= ['DISCHARGE_QTR', 'THCIC_ID', 'PROVIDER_NAME', 'SPEC_UNIT','ENCOUNTER_INDICATOR', 'SEX_CODE', 'TYPE_OF_ADMISSION','SOURCE_OF_ADMISSION', 'PAT_STATE', 'PAT_ZIP', 'PAT_COUNTRY', 'COUNTY','PUBLIC_HEALTH_REGION', 'ADMIT_WEEKDAY', 'LENGTH_OF_STAY', 'PAT_AGE','PAT_STATUS', 'RACE', 'ETHNICITY', 'FIRST_PAYMENT_SRC','SECONDARY_PAYMENT_SRC', 'TYPE_OF_BILL', 'PRIVATE_AMOUNT','CLINIC_AMOUNT', 'TOTAL_CHARGES', 'ADMITTING_DIAGNOSIS','PRINC_DIAG_CODE', 'PRINC_ICD9_CODE', 'RISK_MORTALITY','ILLNESS_SEVERITY', 'ATTENDING_PHYS_UNIF_ID', 'OPERATING_PHYS_UNIF_ID']
    dtypes={'DISCHARGE_QTR':object,'THCIC_ID':int,'PROVIDER_NAME':object,'SPEC_UNIT': object,'ENCOUNTER_INDICATOR':  int,'SEX_CODE':  object,'TYPE_OF_ADMISSION':object,'SOURCE_OF_ADMISSION': object,'PAT_STATE': object,'PAT_ZIP':object,'PAT_COUNTRY':  object,'COUNTY':float,'PUBLIC_HEALTH_REGION':  float,'ADMIT_WEEKDAY': int,'LENGTH_OF_STAY':  float,'PAT_AGE':object,'PAT_STATUS':object,'RACE':object,'ETHNICITY': object,'FIRST_PAYMENT_SRC':object,'SECONDARY_PAYMENT_SRC':  object,'TYPE_OF_BILL': object,'PRIVATE_AMOUNT':  float,'CLINIC_AMOUNT':float,'TOTAL_CHARGES':float,'ADMITTING_DIAGNOSIS': object,'PRINC_DIAG_CODE':  object,'PRINC_ICD9_CODE':  object,'RISK_MORTALITY':int,'ILLNESS_SEVERITY':  int,'ATTENDING_PHYS_UNIF_ID':float,'OPERATING_PHYS_UNIF_ID':float}

    # for each file in the zip folder identify the filenames start with PUDF_base* and read the file into a dataframe 
    for root, dirs, files in os.walk("ZIP"):
        for name in files:
            if("PUDF_base" in name):
                print("Reading file ",name)
                df = pd.read_csv(root+"/"+name,delimiter='\t',usecols = columns,dtype=dtypes)
                data=data.append(df[df['PRINC_DIAG_CODE'].isin(diagnosis_codes)])
    data=data.rename(columns={'THCIC_ID':'PROVIDER_ID'})

Reading file  PUDF_base1q2008_tab.txt
Reading file  PUDF_base2q2008_tab.txt
Reading file  PUDF_base3q2008_tab.txt
Reading file  PUDF_base4q2008_tab.txt
Reading file  PUDF_base1q2009_tab.txt
Reading file  PUDF_base2q2009_tab.txt
Reading file  PUDF_base1q2010_tab.txt
Reading file  PUDF_base2q2010_tab.txt
Reading file  PUDF_base3q2010_tab.txt
Reading file  PUDF_base3q2009_tab.txt
Reading file  PUDF_base4q2009_tab.txt
Reading file  PUDF_base4q2010_tab.txt


If the user has selected to run the notebook with the synthetic data, the diagnosis data is imported from the csv file. The data contains 5 different diagnosis codes, 7453, 74510, 74569, 74511 and 7450.

In [42]:
if data_source == 'synthetic':
    diagnosis_data_file = project.get_file("synthetic diagnosis data.csv")
    diagnosis_data_file.seek(0)
    data = pd.read_csv(diagnosis_data_file)

Once the data has been read into the variable, the next step is to find the hospitals with the highest number of patients. We take the top 15 number of hospitals and filter out all the remaining data from the dataset. 

In [43]:
# Find the count of patients in each hospital 
dfDIAGCODE_count=data.groupby(["PROVIDER_NAME"]).count()['DISCHARGE_QTR'].reset_index()
# Sort the dataframe by count 
dfDIAGCODE_top=dfDIAGCODE_count.sort_values('DISCHARGE_QTR', ascending=False)
# Select top 15 hospital names and store them in a list top_hospitals
top_hospitals=dfDIAGCODE_top.head(15).PROVIDER_NAME.tolist()
# filter the data with the hospital names in the list top_hospitals
dfDIAGCODE_pd=data[data.PROVIDER_NAME.isin(top_hospitals)].copy()
dfDIAGCODE_pd.head()

,DISCHARGE_QTR,PROVIDER_ID,PROVIDER_NAME,SPEC_UNIT,ENCOUNTER_INDICATOR,SEX_CODE,TYPE_OF_ADMISSION,SOURCE_OF_ADMISSION,PAT_STATE,PAT_ZIP,...,PRIVATE_AMOUNT,CLINIC_AMOUNT,TOTAL_CHARGES,ADMITTING_DIAGNOSIS,PRINC_DIAG_CODE,PRINC_ICD9_CODE,RISK_MORTALITY,ILLNESS_SEVERITY,ATTENDING_PHYS_UNIF_ID,OPERATING_PHYS_UNIF_ID
97565,2008Q1,117000,Texas Childrens Hospital,NI,1,M,1,4,TX,77480,...,0.0,70.0,954721.82,74511,74511,3965,4,4,NaN,NaN
99417,2008Q1,117000,Texas Childrens Hospital,I,1,M,3,1,TX,77066,...,12000.0,0.0,104520.41,74511,74511,3921,2,3,1.000000e+10,1.000000e+10
99418,2008Q1,117000,Texas Childrens Hospital,I,1,M,3,1,TX,77067,...,7200.0,0.0,113870.24,74510,74510,3594,2,3,NaN,NaN
99421,2008Q1,117000,Texas Childrens Hospital,I,1,M,3,1,TX,77058,...,4800.0,0.0,98104.33,74569,74569,3512,2,3,NaN,NaN
99427,2008Q1,117000,Texas Childrens Hospital,I,1,F,3,1,TX,77020,...,4800.0,0.0,107231.69,74511,74511,3594,2,3,NaN,NaN


In [44]:
if data_source == 'real':
    # Delete the files once dataset is loaded
    !rm -rf ZIP

The dataset downloaded above includes all the data related to the condition selected by the user. The above dataset contains below columns.<br>
<br>**DISCHARGE_QTR:** Quarter and year in which patient discharged
<br>**PROVIDER_ID:** Hospital id.
<br>**PROVIDER_NAME:** Hospital Name.
<br>**SEX_CODE:** Sex of the patient admitted.
<br>**PAT_STATE:** State in which patient resides.
<br>**PAT_ZIP:** Patients zip code
<br>**LENGTH_OF_STAY:** Length in days patient admitted in the hospital
<br>**PAT_AGE:** Patient's age
<br>**RACE:** Patient's Race
<br>**TOTAL_CHARGES:** Total Charges for the treatment
<br>**PRINC_DIAG_CODE:** Diagnosis code (7453 for common ventricle disease)

In [45]:
dfDIAGCODE_pd.tail()

,DISCHARGE_QTR,PROVIDER_ID,PROVIDER_NAME,SPEC_UNIT,ENCOUNTER_INDICATOR,SEX_CODE,TYPE_OF_ADMISSION,SOURCE_OF_ADMISSION,PAT_STATE,PAT_ZIP,...,PRIVATE_AMOUNT,CLINIC_AMOUNT,TOTAL_CHARGES,ADMITTING_DIAGNOSIS,PRINC_DIAG_CODE,PRINC_ICD9_CODE,RISK_MORTALITY,ILLNESS_SEVERITY,ATTENDING_PHYS_UNIF_ID,OPERATING_PHYS_UNIF_ID
573362,2010Q4,634000,CHRISTUS Santa Rosa Childrens Hospital,N,1,M,2,4,TX,78703,...,0.0,0.0,370818.60,74687,7453,9672,3,4,1.000000e+10,1.000000e+10
594048,2010Q4,686000,Covenant Childrens Hospital,N,1,M,4,5,TX,79424,...,0.0,0.0,19403.16,77089,7450,3891,1,2,1.000000e+10,NaN
713208,2010Q4,852000,Dell Childrens Medical Center,I,1,M,3,1,TX,78723,...,0.0,0.0,111437.12,7453,7453,3594,2,3,1.000000e+10,1.000000e+10
713230,2010Q4,852000,Dell Childrens Medical Center,I,1,M,2,4,TX,78737,...,0.0,0.0,77465.25,7453,7453,NaN,3,4,1.000000e+10,NaN
713232,2010Q4,852000,Dell Childrens Medical Center,NI,1,F,2,4,TX,78612,...,0.0,0.0,483580.00,7453,7453,9547,2,3,1.000000e+10,1.000000e+10


## Data Cleaning and Prep

If using the real data, the payment source and age group are coded and need to be mapped:
    1. Create a dictionary lookup based on the manual https://www.dshs.state.tx.us/thcic/hospitals/UserManual_4q10.pdf for the column FIRST_PAYMENT_SRC.
    2. Create a dictionary lookup based on the manual https://www.dshs.state.tx.us/thcic/hospitals/UserManual_4q10.pdf for the column PRICE_DIAG_CODE.
    
Finally, create a dictionary lookup for the column PRINC_DIAG_CODE. The information is available on http://icd9.chrisendres.com/index.php?action=child&recordid=7440.

In [46]:
# if using thcic data the user needs to map payment and age information. 
# in the synthetic data, this is already done
if data_source == 'real':
    paymentsrc_di={"09": "Self Pay", 
    "HM":"Health Maintenance Organization",
    "10": "Central Certification",
    "LI": "Liability",
    "11": "Other Non-federal Programs" ,
    "LM": "Liability Medical",
    "12": "Preferred Provider Organization (PPO)",
    "MA": "Medicare Part A",
    "13": "Point of Service (POS)", 
    "MB": "Medicare Part B",
    "14": "Exclusive Provider Organization (EPO)",
    "MC": "Medicaid",
    "15": "Indemnity Insurance", 
    "TV": "Title V",
    "16": "Health Maintenance Organization (HMO)",
    "OF": "Other Federal Program",
    "AM": "Automobile Medical",
    "VA": "Veteran Administration Plan",
    "BL": "Blue Cross/Blue Shield", 
    "WC": "Workers Compensation Health Claim",
    "CH": "CHAMPUS", 
    "ZZ": "Charity, Indigent",
    "CI": "Commercial Insurance", 
    "DS": "Disability Insurance", 
    "*":"Invalid",
    "**":"Invalid"}

    #map age group 
    age_di={
    "00":"001-028 days",
    "01":"029-365 days",
    "02":"01-04",
    "03":"05-09",
    "04":"10-14",
    "05":"15-17",
    "06":"18-19",
    "07":"20-24",
    "08":"25-29",
    "09":"30-34",
    "10":"35-39",
    "11":"40-44",
    "12":"45-49",
    "13":"50-54",
    "14":"55-59",
    "15":"60-64",
    "16":"65-69",
    "17":"70-74",
    "18":"75-79",
    "19":"80-84",
    "20":"85-89",
    "21":"90+",
    "*":"Invalid"}

In [47]:
# map icd9 code into diagnosis name
PRINC_DIAG_di={"74569":"74569-Endocard cushion def",
"74511":"74511-Double outlet right ventricle",
"74510":"74510-Complete transp of great vessels",
"7453":"7453-Common ventricle",
"7450":"7450-Common truncus"}

The above dictionaries are used to map the columns in the final datasets 

3 data sets are used for this analysis. <br>

**dfDIAGCODE_pd** : Patient diagnosis information <br>
**dfHospital**: Hospitals location information <br>
**df_patgeo**: Patient location information <br>

Data is prepared and cleaned in sequence of steps to join these data sets and then to calculate the followings.
1. Prepare dfDIAGCODE_pd dataset.
2. Prepare dfHospital dataset.
3. Find the coordinates of the hospitals using dfHospital and join it with dfDIAGCODE_pd using fuzzy match.
4. Find the coordinates of patients using df_patgeo.
5. Calculate the distance between the patient and the hospital that they visited for the treatment using the Levenstein distance. (The Levenshtein distance is a string metric for measuring the difference between two sequences.)
6. Calculate the distance between the patient and the nearest hospital.
7. Check if the hospital the patient visited is same as the nearest hospital.
8. Visualize the final data in terms of distance, cost and severity of the illness etc. 




#### Hospitals that treat this condition

In [48]:
print(dfDIAGCODE_pd.PROVIDER_NAME.drop_duplicates().to_string(index=False))

                    Texas Childrens Hospital
                 St Lukes Episcopal Hospital
             Childrens Medical Center-Dallas
                          Methodist Hospital
               Cook Childrens Medical Center
                Medical City Dallas Hospital
                 Driscoll Childrens Hospital
      CHRISTUS Santa Rosa Childrens Hospital
                 Covenant Childrens Hospital
               Dell Childrens Medical Center
                   Memorial Hermann Hospital
                Providence Memorial Hospital
 UT Southwestern University Hospital-St Paul
     HealthBridge Childrens Hospital-Houston
   Texas Health Presbyterian Hospital Dallas


 #### Diagnosis data cleaning

In [49]:
# Diagnosis data cleaning
dfDIAGCODE_pd['PAT_ZIP'] = pd.to_numeric(dfDIAGCODE_pd['PAT_ZIP'],errors='coerce')
dfDIAGCODE_pd = dfDIAGCODE_pd.dropna(subset=['PAT_ZIP'])
dfDIAGCODE_pd['PAT_ZIP'] = dfDIAGCODE_pd['PAT_ZIP'].astype(int)

In [50]:
dfDIAGCODE_pd['PROVIDER_ID'] = dfDIAGCODE_pd['PROVIDER_ID'].astype(str)
dfDIAGCODE_pd['RACE'] = dfDIAGCODE_pd['RACE'].astype(str)
dfDIAGCODE_pd['ETHNICITY'] = dfDIAGCODE_pd['ETHNICITY'].astype(str)

# Seperate year and quarter into seperate columns , and find charges per day based on total charges and length of stay
dfDIAGCODE_pd['YEAR'] = dfDIAGCODE_pd.DISCHARGE_QTR.str.slice(0,4)
dfDIAGCODE_pd['QTR'] = dfDIAGCODE_pd.DISCHARGE_QTR.str.slice(5,6)
dfDIAGCODE_pd['CHARGES_PER_DAY'] = dfDIAGCODE_pd['TOTAL_CHARGES'] / dfDIAGCODE_pd['LENGTH_OF_STAY'] 

# do some additional cleaning on the thcic data
if data_source == 'real':
    dfDIAGCODE_pd['PROVIDER_NAME'] = dfDIAGCODE_pd['PROVIDER_NAME'].str.strip()
    dfDIAGCODE_pd['PROVIDER_NAME'] = dfDIAGCODE_pd['PROVIDER_NAME'].str.replace('-',' at ')
    dfDIAGCODE_pd['PROVIDER_NAME'] = dfDIAGCODE_pd['PROVIDER_NAME'].str.replace('Rehab','Rehabilitation')
    dfDIAGCODE_pd['PROVIDER_NAME'] = dfDIAGCODE_pd['PROVIDER_NAME'].str.replace('UT ','University of Texas ') 

In [51]:
if data_source == 'real':
    # Update columns based on the above dictionaries
    # 1. Update FIRST_PAYMENT_SRC with paymentsrc_di - replacing codes wit names
    dfDIAGCODE_pd['FIRST_PAYMENT_SRC']=dfDIAGCODE_pd['FIRST_PAYMENT_SRC'].map(paymentsrc_di)
    # 2. AGE_GROUP from PAT_AGE and age_di
    dfDIAGCODE_pd['PAT_AGE']=dfDIAGCODE_pd['PAT_AGE'].map(age_di)

#3. PRINC_DIAG_DESC from PRINC_DIAG_CODE and PRINC_DIAG_di - do this for all data
dfDIAGCODE_pd['PRINC_DIAG_DESC']=dfDIAGCODE_pd['PRINC_DIAG_CODE'].astype(str).map(PRINC_DIAG_di)

###  Hospital Data cleaning

#### Match hospitals names

Above two datasets contain various hospital names. Depending on the source of the data, it is possible that some hospitals have different names in the different datasets. We used fuzzy match from fuzzywuzzy library to join these two datasets. It uses Levenshtein Distance to calculate the differences between sequences.

In [52]:

# Look for the providers name in the hospital data to fetch the longitude and latitude for the data 
# Create an index by providing provider_id, longitude and latitude from dfHospital dataset
choices = dfHospital.set_index(["Provider_ID",'longitude','latitude']).Hospital_Name.to_dict()

# Look for the hospital names in the dfDIAGCODE_pd to find a matching hospital, the returning set contains Provider_name from dfDIAGCODE_pd, Hospital_Name from dfHospital, matching score and the index as an ID
res = [(lookup,) + item for lookup in dfDIAGCODE_pd['PROVIDER_NAME'].unique() for item in process.extract(lookup, choices,limit=1)]
dfHospitalsMatch = pd.DataFrame(res, columns=["PROVIDER_NAME", "Hospital_Name_From_hosp", "Matching_Score", "Combine_ID"])

In [53]:
dfHospitalsMatch

,PROVIDER_NAME,Hospital_Name_From_hosp,Matching_Score,Combine_ID
0,Texas Childrens Hospital,TEXAS CHILDRENS HOSP,91,"(453304, -95.401532, 29.708762)"
1,St Lukes Episcopal Hospital,MAYHILL HOSPITAL,86,"(670010, -97.088293, 33.175842)"
2,Childrens Medical Center at Dallas,CHILDRENS MEDICAL CTR OF DALLAS,89,"(453302, -96.836837, 32.808359)"
3,Methodist Hospital,METHODIST HOSPITAL,100,"(450388, -98.572976, 29.50684)"
4,Cook Childrens Medical Center,COOK CHILDRENS MEDICAL CENTER,100,"(453300, -97.341975, 32.736488)"
5,Medical City Dallas Hospital,MEDICAL CITY DALLAS HOSPITAL,100,"(450647, -96.77208900000001, 32.909447)"
6,Driscoll Childrens Hospital,MAYHILL HOSPITAL,86,"(670010, -97.088293, 33.175842)"
7,CHRISTUS Santa Rosa Childrens Hospital,MAYHILL HOSPITAL,86,"(670010, -97.088293, 33.175842)"
8,Covenant Childrens Hospital,COVENANT CHILDRENS HOSPITAL,100,"(453306, -101.900426, 33.574141)"
9,Dell Childrens Medical Center,DELL CHILDREN'S MEDICAL CENTER OF CENTRAL TEXAS,87,"(453310, -97.704957, 30.302549000000003)"


We decided to exclude the hospital names that have a Levenshtein Distance less than 86.

In [54]:
dfHospitalsMatch=dfHospitalsMatch[dfHospitalsMatch['Matching_Score']>86].copy()
dfHospitalsMatch['PVDR_LONG']=dfHospitalsMatch['Combine_ID'].astype(str).str.split(',').str[1].str.replace("'","")
dfHospitalsMatch['PVDR_LAT']=dfHospitalsMatch['Combine_ID'].astype(str).str.split(',').str[2]
dfHospitalsMatch['PVDR_LAT']=dfHospitalsMatch['PVDR_LAT'].str.replace(")","").str.replace("'","")
dfHospitalsMatch

,PROVIDER_NAME,Hospital_Name_From_hosp,Matching_Score,Combine_ID,PVDR_LONG,PVDR_LAT
0,Texas Childrens Hospital,TEXAS CHILDRENS HOSP,91,"(453304, -95.401532, 29.708762)",-95.401532,29.708762
2,Childrens Medical Center at Dallas,CHILDRENS MEDICAL CTR OF DALLAS,89,"(453302, -96.836837, 32.808359)",-96.836837,32.808359
3,Methodist Hospital,METHODIST HOSPITAL,100,"(450388, -98.572976, 29.50684)",-98.572976,29.50684
4,Cook Childrens Medical Center,COOK CHILDRENS MEDICAL CENTER,100,"(453300, -97.341975, 32.736488)",-97.341975,32.736488
5,Medical City Dallas Hospital,MEDICAL CITY DALLAS HOSPITAL,100,"(450647, -96.77208900000001, 32.909447)",-96.77208900000001,32.909447
8,Covenant Childrens Hospital,COVENANT CHILDRENS HOSPITAL,100,"(453306, -101.900426, 33.574141)",-101.900426,33.574141
9,Dell Childrens Medical Center,DELL CHILDREN'S MEDICAL CENTER OF CENTRAL TEXAS,87,"(453310, -97.704957, 30.302549000000003)",-97.704957,30.302549000000003
10,Memorial Hermann Hospital,MEMORIAL HERMANN NORTHEAST HOSPITAL,95,"(450684, -95.275482, 29.994565)",-95.275482,29.994565
11,Providence Memorial Hospital,MEMORIAL HOSPITAL,90,"(450235, -97.428868, 29.515894999999997)",-97.428868,29.515894999999997
13,HealthBridge Childrens Hospital at Houston,HEALTHBRIDGE CHILDRENS'S HOSPITAL OF HOUSTON,93,"(453309, -95.580593, 29.732317)",-95.580593,29.732317


In [55]:
hospitalTX=dfHospitalsMatch[['PROVIDER_NAME','PVDR_LAT','PVDR_LONG']]

In [56]:
## Add longitude and latitude to the patients zipcode
hospitalTX

,PROVIDER_NAME,PVDR_LAT,PVDR_LONG
0,Texas Childrens Hospital,29.708762,-95.401532
2,Childrens Medical Center at Dallas,32.808359,-96.836837
3,Methodist Hospital,29.50684,-98.572976
4,Cook Childrens Medical Center,32.736488,-97.341975
5,Medical City Dallas Hospital,32.909447,-96.77208900000001
8,Covenant Childrens Hospital,33.574141,-101.900426
9,Dell Childrens Medical Center,30.302549000000003,-97.704957
10,Memorial Hermann Hospital,29.994565,-95.275482
11,Providence Memorial Hospital,29.515894999999997,-97.428868
13,HealthBridge Childrens Hospital at Houston,29.732317,-95.580593


The above dataset now contains the hospital names with longitude and latitude in it.

# Find nearest hospital and distance 
 This function performs following tasks
 1. Assigns latitude and longitude to each patient based on PAT_ZIP.
 2. Calculates the distance travelled by patient to the hospital using longitude and latitudes.
 3. Calculates distance to all the hospitals that treat this condition.
 4. Find the nearest hospital and distance to it.
 5. Checks if the nearest hospital is same as the one patient visited.
 6. Calculates extra distance travelled by the patient.

In [57]:
def find_closest_hospital(df_patgeo,dfDIAGCODE_pd,hospitalTX):
       # Patients data belongs to Texas
    df1=df_patgeo[((df_patgeo['PAT_ZIP'] > 73000) & (df_patgeo['PAT_ZIP'] < 80000)) | ((df_patgeo['PAT_ZIP'] > 88000) & (df_patgeo['PAT_ZIP'] < 89000))] .copy()
    # Copy of diagnosis data 
    df2 = dfDIAGCODE_pd.copy()

    # Create a lookupmatrix to assign longitude and latitude to each record with PAT_ZIP
    dfLookupDistance = pd.merge(df1, df2, how='right',on=['PAT_ZIP'])
    #dfLookupDistance.drop('tmp',1)

    # New dataframe by joining hospital data and lookup data on provider name
    dfTXPTNT2Hospitals = pd.merge(hospitalTX, dfLookupDistance, how = 'left', on='PROVIDER_NAME')

    # Drop nulls 
    dfTXPTNT2Hospitals = dfTXPTNT2Hospitals.dropna(subset=['PVDR_LAT'])
    dfTXPTNT2Hospitals = dfTXPTNT2Hospitals.dropna(subset=['PVDR_LONG'])
    dfTXPTNT2Hospitals = dfTXPTNT2Hospitals.dropna(subset=['PAT_LAT'])
    dfTXPTNT2Hospitals = dfTXPTNT2Hospitals.dropna(subset=['PAT_LONG'])

    # Calculate distance between the patient and the hospital they were admitted to 
    dfTXPTNT2Hospitals['DISTANCE'] = dfTXPTNT2Hospitals.apply(lambda x: geodesic((x['PVDR_LAT'], x['PVDR_LONG']), (x['PAT_LAT'], x['PAT_LONG'])).miles, axis = 1)
    Diagdata=dfTXPTNT2Hospitals.copy()
    hospitaldata = hospitalTX.copy()
    # Cross Join Diagnosis data and hospital data to get all possible combinations
    Diagdata['tmp']=1
    hospitaldata['tmp']=1
    
    crs_data=pd.merge(Diagdata,hospitaldata,on='tmp')
    
    # Create a new column to calculate distance to all the hospitals using longitude and latitude
    crs_data['DISTANCE_TO_HOSPITAL'] = crs_data.apply(lambda x: geodesic((x['PVDR_LAT_y'], x['PVDR_LONG_y']), (x['PAT_LAT'], x['PAT_LONG'])).miles, axis = 1)
    
    # Check the hospital visited by the patient is nearest or not
    crs_data['IS_NEAREST'] = crs_data.groupby(['PAT_ZIP'])['DISTANCE_TO_HOSPITAL'].transform(min) == crs_data['DISTANCE_TO_HOSPITAL']
    
    # Find the distance to the nearest hospital
    crs_data['MIN_DISTANCE']=crs_data.groupby(['PAT_ZIP'])['DISTANCE_TO_HOSPITAL'].transform(min)
    
    # Create a dataset with the nearest hospital only
    PROV2ZIP = crs_data.loc[crs_data.groupby('PAT_ZIP')['DISTANCE_TO_HOSPITAL'].idxmin()]
    MIN_PROV2ZIP = pd.DataFrame(data=PROV2ZIP,columns=['PAT_ZIP','PROVIDER_NAME_y'])
    MIN_PROV2ZIP= MIN_PROV2ZIP.rename(columns={"PROVIDER_NAME_y": "NEAREST_HOSPITAL"})
    
    
    # Create another dataset with unique combination of minimum distance and join it with crossjoin data
    NEAREST_HOSPITAL_DISTANCE = crs_data.groupby('PAT_ZIP')["DISTANCE_TO_HOSPITAL"].min().rename("NEAREST_HOSPITAL_DISTANCE").reset_index()
    allDist_PP_0 = crs_data.merge(NEAREST_HOSPITAL_DISTANCE)
    
    # Join above two datasets on PAT_ZIP
    allDist_PP_1 = pd.merge(allDist_PP_0, MIN_PROV2ZIP, how = 'left', on='PAT_ZIP')
    
    #Create a new column 'TRADEOFF' to calculate extra distance travelled by the patient
    allDist_PP_1['TRADEOFF'] = allDist_PP_1['DISTANCE_TO_HOSPITAL'] - allDist_PP_1['NEAREST_HOSPITAL_DISTANCE']
    
    allDist_PP_1=allDist_PP_1[round(allDist_PP_1['DISTANCE'],3)==round(allDist_PP_1['DISTANCE_TO_HOSPITAL'],3)]
    #Remove duplicates
    allDist_PP_1= allDist_PP_1.drop_duplicates()
    # Create a dataset with selected columns
    Tradeoff_matrix = allDist_PP_1[['PAT_ZIP','PROVIDER_NAME_x', 'DISTANCE_TO_HOSPITAL', 'IS_NEAREST','NEAREST_HOSPITAL_DISTANCE','NEAREST_HOSPITAL','TRADEOFF','PVDR_LONG_x','PVDR_LAT_x','PAT_LONG','PAT_LAT']].copy()
    Tradeoff_matrix['PROVIDER_NAME'] = Tradeoff_matrix['PROVIDER_NAME_x'].astype(str)
    
    #Create a final datasets by 
    AllPatients = pd.merge(dfDIAGCODE_pd, Tradeoff_matrix, how = 'left', on=['PAT_ZIP','PROVIDER_NAME']) 
    AllPatients=AllPatients.drop_duplicates()
    AllPatients = AllPatients[pd.notnull(AllPatients['NEAREST_HOSPITAL_DISTANCE'])]
    AllPatients=AllPatients.rename(columns={"PVDR_LONG_x": "PVDR_LONG", "PVDR_LAT_x": "PVDR_LAT"})
    return AllPatients

In [58]:
# Create a blank dataframe
AllPatients=pd.DataFrame()

# For each diagnosis code in dfDIAGCODE_pd call the function find_closest_hospital() and append it to the dataframe
for code in dfDIAGCODE_pd["PRINC_DIAG_CODE"].unique().tolist():
    AllPatients=AllPatients.append(find_closest_hospital(df_patgeo,dfDIAGCODE_pd[dfDIAGCODE_pd["PRINC_DIAG_CODE"]==code],hospitalTX))

The patient counts for each diagnosis code

In [59]:
dfDIAGCODE_pd["PRINC_DIAG_CODE"].value_counts()



74569    487
74511    327
74510    191
7453     164
7450     108
Name: PRINC_DIAG_CODE, dtype: int64

The final data set contains the information on the hospitals for the diagnosis selected by the users. It includes patient's personal details, distance travelled by the patients to the hospital, nearest hospital to the patient, cost of hospital stay, length of stay before discharging and total charges.

The final dataset contains below columns.

<br>**DISCHARGE_QTR:** Quarter and year in which patient discharged
<br>**PROVIDER_ID:** Hospital id.
<br>**PROVIDER_NAME:** Hospital Name.
<br>**PVDR_LONG:** Hospital Longitude
<br>**PVDR_LAT:** Hospital Latitude
<br>**PAT_LONG:** Patient Longitude
<br>**PAT_LAT:** Patient Latitude
<br>**SEX_CODE:** Sex of the patient admitted.
<br>**PAT_STATE:** State in which patient resides.
<br>**PAT_ZIP:** Patient's zip code
<br>**LENGTH_OF_STAY:** Length in days patient stayed in the hospital
<br>**PAT_AGE:** Patient's age
<br>**RACE:** Patient's Race
<br>**TOTAL_CHARGES:** Total Charges for the treatment
<br>**PRINC_DIAG_CODE:** Principle Diagnosis code 
<br>**CHARGES_PER_DAY:** Patient charges per day
<br>**DISTANCE_TO_HOSPITAL:** Distance travelled by the patient to the hospital
<br>**NEAREST_HOSPITAL:** Name of the nearest hospital
<br>**IS_NEAREST:** Boolean variable to identify if the Hospital visited by the patient is nearest.
<br>**NEAREST_HOSPITAL_DISTANCE:** Distance to the nearest hospital.
<br>**TRADEOFF:** Additional distance travelled by the patient.
<br>**FIRST_PAYMENT_SRC:** Payment source description.
<br>**PAT_AGE:** PAT_AGE group
<br>**PRINC_DIAG_DESC:** Descriptions for PRINC_DIAG_CODE.

### Save the dataset to build dashboard

In [60]:
project.save_data("AllPatients.csv", AllPatients.to_csv(index=False), overwrite=True)

{'file_name': 'AllPatients.csv',
 'message': 'File saved to project storage.',
 'asset_id': '0b1f7bfb-8384-4d53-aef4-437077d24c6d'}

In [61]:
AllPatients.columns

Index(['DISCHARGE_QTR', 'PROVIDER_ID', 'PROVIDER_NAME', 'SPEC_UNIT',
       'ENCOUNTER_INDICATOR', 'SEX_CODE', 'TYPE_OF_ADMISSION',
       'SOURCE_OF_ADMISSION', 'PAT_STATE', 'PAT_ZIP', 'PAT_COUNTRY', 'COUNTY',
       'PUBLIC_HEALTH_REGION', 'ADMIT_WEEKDAY', 'LENGTH_OF_STAY', 'PAT_AGE',
       'PAT_STATUS', 'RACE', 'ETHNICITY', 'FIRST_PAYMENT_SRC',
       'SECONDARY_PAYMENT_SRC', 'TYPE_OF_BILL', 'PRIVATE_AMOUNT',
       'CLINIC_AMOUNT', 'TOTAL_CHARGES', 'ADMITTING_DIAGNOSIS',
       'PRINC_DIAG_CODE', 'PRINC_ICD9_CODE', 'RISK_MORTALITY',
       'ILLNESS_SEVERITY', 'ATTENDING_PHYS_UNIF_ID', 'OPERATING_PHYS_UNIF_ID',
       'YEAR', 'QTR', 'CHARGES_PER_DAY', 'PRINC_DIAG_DESC', 'PROVIDER_NAME_x',
       'DISTANCE_TO_HOSPITAL', 'IS_NEAREST', 'NEAREST_HOSPITAL_DISTANCE',
       'NEAREST_HOSPITAL', 'TRADEOFF', 'PVDR_LONG', 'PVDR_LAT', 'PAT_LONG',
       'PAT_LAT'],
      dtype='object')

In [62]:
AllPatients_agg=AllPatients.groupby(['PROVIDER_ID',"PROVIDER_NAME",'PVDR_LONG','PVDR_LAT']).count()['DISCHARGE_QTR'].reset_index()
AllPatients_agg.columns=["PROVIDER_ID","PROVIDER_NAME","PVDR_LONG","PVDR_LAT","PATIENT_COUNT"]
project.save_data("HospitalTreatCondition.csv", AllPatients_agg.to_csv(index=False), overwrite=True)

{'file_name': 'HospitalTreatCondition.csv',
 'message': 'File saved to project storage.',
 'asset_id': '5aba705e-00e3-4303-9d5b-5154a067b4ac'}

***Next we can move to app-deployment notebook, where we will go through the steps to deploy the Shiny dashboard app.*** 